<a href="https://colab.research.google.com/github/lgiesen/forest-height/blob/main/21_decision_tree_regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import drive
drive.mount ('/content/drive', force_remount=True)
root_path = 'drive/MyDrive/Colab Notebooks/data/'

Mounted at /content/drive


In [25]:
%run "/content/drive/MyDrive/Colab Notebooks/0_load_data.ipynb"

Mounted at /content/drive
0_load_data.ipynb completed.


In [26]:
# load dataset
path_train_sat = f"{root_path}train_satellite.npy"
path_train_masks = f"{root_path}train_masks.npy"

X = np.load(path_train_sat, allow_pickle=True)
y = np.load(path_train_masks, allow_pickle=True)

In [27]:
# remove drive connection as it is no longer needed
drive.flush_and_unmount()

In [28]:
# ceil the values at 2000 because clouds have a different reflection value
ceiling = 2000
X[X > ceiling] = ceiling

In [29]:
# scale values between 0 and 1
X = X/ceiling

In [30]:
X.shape

(20, 10, 1024, 1024)

# Creating Cutouts

In [50]:
patch_shape = (1, 256, 256, 10) # needs to have the same number of dimensions as the whole image
step_size = 256

In [ ]:
!pip install patchify

In [65]:
from patchify import patchify, unpatchify
image = np.reshape(X, (20, 1024, 1024, 10))
patches = patchify(image, patch_shape, step=step_size)
print(patches.shape)

(1, 4, 4, 1, 1, 256, 256, 10)


In [66]:
X.shape, image.shape

((20, 10, 1024, 1024), (20, 1024, 1024, 10))

In [72]:
assert patches.shape == (1, 4, 4, 1, 1, 256, 256, 10)
reconstructed_image = unpatchify(patches, image.shape)
print(reconstructed_image.shape)

assert (reconstructed_image == image).all()

NotImplementedError: ignored

Undo patchify (merge cutouts to one larger image) with: `unpatchify(patches_to_merge, merged_image_size` => `reconstructed_image = unpatchify(patches, image.shape)`

Split the dataset into training and test set

In [ ]:
from sklearn.model_selection import train_test_split
# split dataset for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

In [ ]:
del X
del y

In [ ]:
X_train.shape, type(X_train), y_train.shape, type(y_train)

# Data Augmentation

In [ ]:
import tensorflow as tf
from tensorflow import keras
#Data augmentation
data_augmentation = tf.keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    #keras.layers.experimental.preprocessing.RandomRotation(0.4),
    #keras.layers.RandomCrop(224, 224)
])

image = scaled_img.reshape((3, 1024, 1024))
image = np.asarray(scaled_img)

In [ ]:
#image = tf.cast(tf.expand_dims(image, 0), tf.float32)
plt.figure(figsize=(10, 10))

for i in range(9):
  augmented_image = data_augmentation(image, training=True)
  ax = plt.subplot(3, 3, i + 1)
  plt.subplots_adjust(hspace=0.07, wspace=0.01)
  plt.imshow(augmented_image)
  plt.axis("off")
  #print(augmented_image.shape)
  #print(i)
  #plot_img(augmented_image)

## Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# instantiate the model
# TODO: Try out different values for min_samples_split
# corresponds to "m" in the slides
min_samples_split = 2

# instantiate model and fit it!
model = DecisionTreeRegressor(max_depth=5, min_samples_split=min_samples_split)

In [ ]:
def reshape_2D(var):
  if len(var.shape) == 4:
    nsamples, color_channel, nx, ny = var.shape
  else:
    nsamples, nx, ny = var.shape
    color_channel = 1
  return var.reshape((nsamples, color_channel*nx*ny))

In [ ]:
# reshape data so that DecisionTreeRegressor can work with the data
X_train_2D = reshape_2D(X_train).tolist()
y_train_2D = reshape_2D(y_train).tolist()

> Try out df as a data format. Previously tried out: ndarray, list

> Use k-fold cross-validation for hyperparameter tuning of `max_depth`

In [ ]:
del X, y

In [ ]:
%%time
# train the model
model.fit(X_train_2D, y_train_2D)
#model_2.fit(X_train_2D, y_train_2D)

In [ ]:
from sklearn.model_selection import GridSearchCV
# k-fold cross-validation for hyperparameter tuning of max_depth
gs = GridSearchCV(model,
                  param_grid = {'max_depth': range(1, 11),
                                'min_samples_split': range(10, 60, 10)},
                  cv=5,r
                  n_jobs=1,
                  scoring='neg_mean_squared_error')

gs.fit(X_train, y_train) # X_train_2D, y_train_2D

print(gs.best_params_)
print(-gs.best_score_)

In [ ]:
# Predict
# X_test = np.arange(0.0, 5.0, 0.01)[:, np.newaxis]
X_test_2D = reshape_2D(X_test).tolist()
y_test_2D = reshape_2D(y_test).tolist()
preds = model.predict(X_test_2D)
#preds_model_2 = model_2.predict(y_test_2D)

In [ ]:
model.summary()

In [ ]:
# save the model to disk
import pickle
path_reg_model = root_path + 'DecisionTreeRegressor.sav'
pickle.dump(model, open(path_reg_model, 'wb'))
# loaded_model = pickle.load(open(path_reg_model, 'rb'))

In [ ]:
# save the model to disk
import pickle
from sklearn.externals import joblib
path_reg_model = root_path +  "DecisionTreeRegressor.joblib"
joblib.dump(model, path_reg_model)
# loaded_model = joblib.load(path_reg_model)

In [ ]:
# Plot the results
plt.figure()
plt.scatter(X, y, s=20, edgecolor="black", c="darkorange", label="data")
plt.plot(X_test_2D, y_test_2D, color="cornflowerblue", label="max_depth=2", linewidth=2)
plt.plot(X_test_2D, y_test_2D, color="yellowgreen", label="max_depth=5", linewidth=2)
plt.xlabel("data")
plt.ylabel("target")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()

# from the Lecture:
X_plot = np.arange(0.0, 5.0, 0.01).reshape((-1,1))
plt.figure(figsize=(10,6))
plt.scatter(X, y, c="darkorange", label="Data", s=25)
plt.plot(X_plot, preds, color="cornflowerblue", label="Model", linewidth=3)
plt.xlabel("X", fontsize=18)
plt.ylabel("Y", fontsize=18)
plt.title(f"Regression Tree (m={min_samples_split})", fontsize=16)
plt.legend()

#
sns.scatterplot(x=df['Longitude'],
                y=df['MedHouseVal'],
                label='data')

plt.plot(df['Longitude'].sort_values(),
         dtr1.predict(df['Longitude'].sort_values().to_frame()),
         color='red', label='model',
         linewidth=2)

plt.legend()

plt.savefig('model.png')

In [ ]:
# visualize tree
from sklearn.tree import export_graphviz
import graphviz

dot_data = export_graphviz(dtr1, feature_names=['Longitude'],
                           filled=True, rounded=True)

graph = graphviz.Source(dot_data)
graph.render("tree")

Visualize Decision Tree: https://towardsdatascience.com/visualizing-decision-trees-with-python-scikit-learn-graphviz-matplotlib-1c50b4aa68dc

In [ ]:
# Evaluate Decision Tree Regressor
loss, acc = model.evaluate(X_test_2D, y_test_2D, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))